## **Kompletna Aplikacja REST**

### **Wstęp**

Projekt 3 zakłada zaprogramowanie aplikacji TODO. Aplikacja powinna zawierać/obsługiwać następujące elementy:

- Baza SQL - obsługa rekordów i tabel niezbędnych do przechowywania informacji o zadaniach.
- Uwierzytelnianie - możliwość tworzenia użytkowników, tj. rejestracja i logowanie.
- Autoryzacja - role i obowiązki w których użytkownik może być administratorem, np. dostęp do niektórych endpoint'ów.

Aplikacja powinna obsługiwać stronę internetową do komunikacji z serwerem FastAPI oraz bazę danych do komunikacji z samą aplikacją. 

### **Baza Danych SQLite**


In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

SQLAlchemy to ORM, czyli obiektowe mapowanie relacyjne, z którego będzie korzystać nasza aplikacja Fast API, aby móc utworzyć bazę danych i utworzyć połączenie z bazą danych oraz mieć możliwość korzystania ze wszystkich rekordów bazy w naszej aplikacji.

Adres `SQLALCHEMY_DATABASE_URL` zostanie użyty do utworzenia lokalizacji bazy danych w aplikacji Fast API. 

Funkcja `create_engine()` tworzy silnik `engine` aby móc utworzyć połączenie z bazą. Argument `check_same_thread=False` jest istotny dla baz danych SQLite i pozwala na współdzielenie połączenia z bazą danych przez wiele wątków. Domyślnie połączenia SQLite są jednowątkowe, a to ustawienie zastępuje to zachowanie.

Funkcja `sessionmaker()` jest używana do utworzenia fabryki sesji, która jest konfigurowalną fabryką do tworzenia nowych obiektów `Session`. Tutaj `autocommit=False` zapewnia, że transakcje nie są automatycznie zatwierdzane, wymagając wyraźnych zatwierdzeń. Argument `autoflush=False` wyłącza automatyczny przepływ zmian do bazy danych, dając większą kontrolę nad tym, kiedy zmiany są wysyłane. Argument `bind=engine` wiąże sesję z wcześniej utworzonym silnikiem, zapewniając, że wszystkie sesje utworzone przez tę fabrykę będą korzystać z tego samego połączenia z bazą danych.

Wreszcie, `Base = declarative_base()` tworzy klasę bazową dla deklaratywnych definicji klas. Ta klasa `Base` będzie używana jako baza dla wszystkich klas mapowanych ORM, umożliwiając im dziedziczenie wspólnych metadanych i metod.

Razem, komponenty te tworzą podstawową infrastrukturę dla operacji bazodanowych, umożliwiając tworzenie połączeń bazodanowych, sesji dla transakcji i klas mapowanych ORM do interakcji z tabelami bazy danych.

In [2]:
from sqlalchemy import Boolean, Column, Integer, String


class Todos(Base):
    __tablename__ = "todos"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    priority = Column(Integer)
    complete = Column(Boolean, default=False)

Model `Todos` reprezentuje tabelę w relacyjnej bazie danych, gdzie każda instancja klasy `Todos` odpowiada wierszowi w tabeli. Atrybut `__tablename__` określa nazwę tabeli w bazie danych. W klasie `Todos` zdefiniowanych jest kilka kolumn przy użyciu klasy `Column` SQLAlchemy. Każda kolumna odpowiada polu w tabeli bazy danych.

In [3]:
from fastapi import FastAPI

app = FastAPI()
Base.metadata.create_all(bind=engine)

Ten fragment kodu konfiguruje aplikację FastAPI i zapewnia utworzenie niezbędnych tabel bazy danych za pomocą SQLAlchemy. 
Metoda `create_all` jest wywoływana w celu utworzenia wszystkich tabel w bazie danych, które są zdefiniowane przez modele. Argument `bind=engine` określa silnik bazy danych, który ma być używany do tworzenia tabel.

### **Obsługa Bazy Danych SQLite**

W celu obsługi bazy danych SQLite możemy zainstalować pakiet narzędzi wiersza poleceń -> https://www.sqlite.org/download.html. Następnie uruchamiamy następujące polecenie:

```bash
sqlite3
```

### **FastAPI - Obsługa Tabel**

In [4]:
from typing import Annotated

from fastapi import Depends
from sqlalchemy.orm import Session
from starlette import status


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


@app.get("/", status_code=status.HTTP_200_OK)
async def read_all(db: Annotated[Session, Depends(get_db)]):
    return db.query(Todos).all()

Funkcja `read_all()` przyjmuje pojedynczy parametr `db`, który jest opatrzony adnotacją `Annotated[Session, Depends(get_db)]`. Adnotacja ta wskazuje, że `db` jest zależnością, która powinna być dostarczona przez funkcję `get_db()`. Funkcja `Depends` jest narzędziem dostarczanym przez FastAPI do deklarowania zależności, a `get_db()` jest funkcją, która konfiguruje i dostarcza sesję bazy danych SQLAlchemy. Wewnątrz funkcji, `db.query(Todos).all()` wywołuje zapytanie do bazy danych o wszystkie rekordy w tabeli `Todos`.

Tak więc jesteśmy w stanie pobrać wszystkie informacje z naszej bazy danych, ponieważ używamy wstrzykiwania zależności aby najpierw pobrać i uruchomić `get_db()`. Sprawia to, że nasza baza danych ma teraz sesję lokalną, co tak naprawdę oznacza, że jesteśmy w stanie skontaktować się z bazą danych.

### **Uwierzytelnianie & Autoryzacja**

Jaka jest różnica między uwierzytelnianiem a autoryzacją? 

**Uwierzytelnianie potwierdza, że użytkownicy są tymi, za których się podają.** 

**Autoryzacja daje tym użytkownikom pozwolenie na dostęp do zasobu.**

In [9]:
import bcrypt

password = "password"
hashed_password = bcrypt.hashpw(password.encode("utf-8"), bcrypt.gensalt())

bcrypt.checkpw(password.encode("utf-8"), hashed_password)

True

Importujemy bibliotekę `bcrypt`, która jest popularną biblioteką do hashowania haseł. Zapewnia ona funkcje do bezpiecznego hashowania i weryfikacji haseł przy użyciu algorytmu hashowania bcrypt.

Następnie hashujemy hasło przy użyciu funkcji `hashpw()`. Aargument `bcrypt.gensalt()` generuje wartość losową, która będzie dodana do hasła przed haszowaniem, aby zapewnić, że nawet identyczne hasła będą miały różne skróty.




### **JSON Web Token (JWT)**

JWT jest jednym z najpopularniejszych tokenów na okaziciela (tzw. bearer token) i protokołów autoryzacji w interfejsach API. JSON Web Token wyróżnia się następującymi cechami:

- To samodzielny sposób na bezpieczne przesyłanie danych i informacji między dwiema lub więcej stronami przy użyciu obiektu JSON.
- Może być zaufany, ponieważ każdy JWT może być podpisany cyfrowo, co zasadniczo pozwala serwerowi wiedzieć, czy obiekt JSON został w ogóle zmieniony przez klienta.
- Nie jest metodą uwierzytelniania ale metodą autoryzacji, która pozwala klientowi i serwerowi utrzymywać relację bez konieczności logowania się przy każdym żądaniu.

JWT jest zbudowany z trzech części odseparowanych za pomocą znaku kropki, np. `aaa.bbb.ccc`. Elementy te to header (a), payload (b) oraz signature (c).

Więcej informacji na https://jwt.io/.